In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def load_video(path, ms=100):
    
    # Video Capture 객체 생성
    capture = cv2.VideoCapture(path)
    
    # Frame을 저장할 List 선언
    frames = []
    
    while capture.isOpened(): # Video Capture가 준비되었는지 확인
        
        run, frame = capture.read() # 다음 Frame 읽기
        # ROI 설정 (x, y, width, height)
        roi_x, roi_y, roi_w, roi_h = 0, 280, 852, 200
        
        # ROI 추출
        roi = frame[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w]

        # 그레이스케일 변환
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)
        # 원본 프레임에 엣지 결과를 덮어쓰기 (ROI 영역만)
        frame[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w] = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        
        
        if run: # Frame을 읽은 경우
            # 결과 출력
            cv2.imshow('Frame with ROI Edges', frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else: # 재생이 완료되어 더 이상 Frame을 읽을 수 없는 경우
            break
        
        # Frame List에 추가
        frames.append(frame)

    capture.release() # Capture 자원 반납
    cv2.destroyAllWindows() # 창 제거
    
    return np.array(frames, dtype='uint8')

In [4]:
# Video가 저장된 경로 입력
PATH = r".\plank_sample2.mp4"
# PATH = r"C:\Users\admin\Desktop\KUDIP\Video Samples\Hand Video2.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\highway.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\earth.avi"

# Video 재생 및 반환 (Numpy Array)
video = load_video(PATH)

In [5]:
print(f"- Data Type: {type(video)}")
print(f"- Data Shape: {video.shape} *Frames x Height x Width x Channel")
print(f"- Maximum Intensity: {video.max()}")
print(f"- Minimum Intensity: {video.min()}")

- Data Type: <class 'numpy.ndarray'>
- Data Shape: (261, 480, 852, 3) *Frames x Height x Width x Channel
- Maximum Intensity: 255
- Minimum Intensity: 0


In [6]:
import cv2
import numpy as np

def high_boost_filtering_bgr(image, kernel_size, alpha):
    # 이미지 각 채널에 대해 고역통과 필터링 적용
    b, g, r = cv2.split(image)
    
    # 각 채널에 대해 고역통과 필터링 수행
    b_filtered = high_boost_filtering_channel(b, kernel_size, alpha)
    g_filtered = high_boost_filtering_channel(g, kernel_size, alpha)
    r_filtered = high_boost_filtering_channel(r, kernel_size, alpha)

    # 고역통과 필터링된 이미지 병합
    result = cv2.merge((b_filtered, g_filtered, r_filtered))

    return result

def high_boost_filtering_channel(channel, kernel_size, alpha):
    # 라플라시안 필터 생성
    laplacian_kernel = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]])

    # 채널에 라플라시안 필터 적용
    laplacian = cv2.filter2D(channel, -1, laplacian_kernel)

    # 고역통과 필터링된 이미지 생성
    high_boost = channel + alpha * laplacian

    # 이미지 범위를 0~255로 조정
    high_boost = np.clip(high_boost, 0, 255)
    high_boost = high_boost.astype(np.uint8)

    return high_boost


In [25]:
import cv2
import numpy as np

# 영상 읽기
cap = cv2.VideoCapture('plank_sample2.mp4')

def get_centroid(contour):
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        return (cX, cY)
    else:
        return (0, 0)
    
while True:
    ret, frame = cap.read()
    # ROI 설정 (x, y, width, height)
    roi_x, roi_y, roi_w, roi_h = 0, 280, 852, 200
    
    # ROI 추출
    roi = frame[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w]

    if not ret:
        break

    # 그레이스케일 변환 및 가우시안 블러 적용
    
    kernel_size = 3
    alpha = 1.5
    result = high_boost_filtering_bgr(roi, kernel_size, alpha)
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    median_blurred_image = cv2.medianBlur(gray, 3)

    # Laplacian 필터를 적용하여 에지를 검출합니다.
    log_edges = cv2.Laplacian(median_blurred_image, cv2.CV_64F)

    # 결과 이미지를 8비트로 변환합니다.
    log_edges = np.uint8(np.absolute(log_edges))
    
    cv2.imshow('log_edges', log_edges)

    # # 객체 추출
    # contours, _ = cv2.findContours(log_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # # 상체와 하체의 중심점 초기화
    # upper_body_contours = []
    # lower_body_contours = []

    # # 프레임의 중앙을 기준으로 상체와 하체를 나눔
    # roi_height = roi.shape[0]
    # roi_center_y = roi_height // 2

    # # 컨투어 처리
    # for contour in contours:
    #     area = cv2.contourArea(contour)
    #     if area > 1000:  # 노이즈 제거를 위한 최소 면적 설정
    #         x, y, w, h = cv2.boundingRect(contour)
    #         centroid = get_centroid(contour)
    #         if centroid[1] < roi_center_y:  # 상체로 간주
    #             upper_body_contours.append(contour)
    #         else:  # 하체로 간주
    #             lower_body_contours.append(contour)

    # # 상체와 하체의 컨투어 그리기
    # cv2.drawContours(roi, upper_body_contours, -1, (0, 255, 0), 2)  # 초록색: 상체
    # cv2.drawContours(roi, lower_body_contours, -1, (255, 0, 0), 2)  # 파란색: 하체

    # 결과 출력
    # cv2.imshow('Canny', edges)
    # cv2.imshow('Plank Pose - Upper and Lower Body', roi)
    # cv2.imshow('otsu', t_otsu)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [31]:
import cv2
import numpy as np

def background_subtraction(frame, fgbg):
    fgmask = fgbg.apply(frame)
    return fgmask

def detect_largest_contour(fgmask):
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        return largest_contour
    return None

def draw_contour(frame, contour, color=(0, 255, 0)):
    cv2.drawContours(frame, [contour], -1, color, 2)

def get_contour_center(contour):
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        return (cx, cy)
    return None

# 비디오 캡처 초기화
cap = cv2.VideoCapture('plank_sample2.mp4')
fgbg = cv2.createBackgroundSubtractorMOG2()

# 기준선 설정 (예제에서는 화면 중앙)
ret, frame = cap.read()
frame_height, frame_width = frame.shape[:2]
baseline = 100

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    roi_x, roi_y, roi_w, roi_h = 0, 280, 852, 200
    
    # ROI 추출
    frame = frame[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w]
    
    fgmask = background_subtraction(frame, fgbg)
    largest_contour = detect_largest_contour(fgmask)

    if largest_contour is not None:
        contour_center = get_contour_center(largest_contour)
        
        if contour_center:
            cx, cy = contour_center
            # 엉덩이 위치를 기준으로 위 또는 아래로 벗어났는지 확인
            if cy < baseline - 30:
                color = (0, 0, 255)  # 빨간색: 너무 올라감
                message = "Too High"
            elif cy > baseline + 30:
                color = (255, 0, 0)  # 파란색: 너무 내려감
                message = "Too Low"
            else:
                color = (0, 255, 0)  # 초록색: 올바른 위치
                message = "Correct"
                
            draw_contour(frame, largest_contour, color)
            cv2.circle(frame, (cx, cy), 5, color, -1)
            cv2.putText(frame, message, (cx, cy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # 기준선을 그립니다.
    cv2.line(frame, (0, baseline), (frame_width, baseline), (255, 255, 255), 2)
    
    cv2.imshow('Plank Pose Detection', frame)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
